# Interactive Look at How EV Charging Affects California's Duck Curve

This notebook is a deeper look at a recent Medium article that I published on [energy efficient technologies](https://medium.com/age-of-awareness/energy-efficiency-should-be-publicized-more-than-renewables-7e5e2192d0d6) and the potential reductions in carbon emissions that they can provide today.

I want to further explore California's duck curve and electric vehicle (EV) charging. As explained in the article, the duck curve is a pattern of electricity demand that occurs because solar energy reduces the grid's net load, which is the amount of power the grid system operator needs to provide from non-renewable sources, specifically it is **total demand** minus **power provided by solar & wind**.

In my article I mentioned that electric vehicle charging will be an additional load that the California grid operator will need to supply, ideally with non-carbon emitting energy sources. I want to visualize what the additional demand from EV charging may look like and how much additional carbon may be emitted from commuters charging at peak load hours.

***
IMPORTANT: If you are unfamiliar with Jupyter Notebooks and encounter an error, press the &#x25B6;&#x25B6; button in the toolbar above to restart and run the code blocks.
***

## Table of Contents
- [Loading Data](#loading-data)
- [Base Duck Curve Chart](#base-chart)
- [Carbon Emissions](#carbon-emissions)
    - [Unexpected result in rising Natural Gas and Natural Gas CO<sub>2</sub> emissions per MW](#natural-gas-peak-emissions)
- [Charging More Electric Vehicles](#electric-vehicles)
    - [How do Additional EVs Affect Grid CO<sub>2</sub> Release](#ev-grid-release)
    - [How to Approach Additional Grid Demand From EVs](#approach-grid-demand)
- [Takeaways](#takeaways)

### <a name="loading-data"></a> Loading Data
Not much to this section. Just loading and cleaning up data from the California Independent System Operator [CAISO](http://www.caiso.com/TodaysOutlook/Pages/default.aspx). The dataset is taken from March 1, 2020 as a prototypical example of the 'duck curve' and because March is a month with greater than average [solar curtailment](http://www.caiso.com/informed/Pages/ManagingOversupply.aspx#dailyCurtailment), likely due to reduced electricity demand in cooler winter months relative to a/c heavy summer months.

In [1]:
import pandas as pd

loadDf = pd.read_excel('data/CAISO-hourly-data.xlsx', index_col=0)
loadDf = loadDf.transpose().drop(['Hour ahead forecast', 'Sum of Renewables', 'Other'], axis=1)

emissionsDf = pd.read_excel('data/CAISO-hourly-emissions-data.xlsx', index_col=0)
emissionsDf = emissionsDf.transpose()

fullDf = loadDf.merge(emissionsDf, left_index=True, right_index=True, suffixes=('','_CO2'))

# Get data as arrays
xValues = fullDf.index.values.tolist()
xValues = list(map(lambda x: x.strftime("%H:%M"), xValues))

yDemand = fullDf['Demand (5 min. avg.)'].tolist()
yNetDemand = fullDf['Net demand'].tolist()
ySolarSupply = fullDf['Solar'].tolist()
yWindSupply = fullDf['Wind'].tolist()
yCoalSupply = fullDf['Coal'].tolist()
yGasSupply = fullDf['Natural gas'].tolist()

### <a name="base-chart"></a> Base Duck Curve Chart
Here we are plotting the CAISO data for every 5 minutes of March 1, 2020. In particular, we are looking at the demand (or load) curves for the duck curve trend juxtaposed against the supply curves from large renewable sources, solar and wind, as well as fossil fuels that the state is trying to move away from.

We can see the duck curve in the chart line labeled **Net demand**, which is the total electricity demand - solar & wind. As the sun comes up around 7 am, solar power production shoots up to a peak of just over 7,000 MW by 10 am, which is just over a third of the state's total demand during daytime.

The state's transition to solar has been a bright spot in the limiting climate change. When it comes to reducing fossil fuels the state still relies heavily on natural gas, but coal provides very little energy at least in cooler winter months like March.

In [2]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create figure with secondary y-axis
hourlyFigure = make_subplots()
    
# Create traces
demandTrace = go.Scatter(x=xValues, y=yDemand, name="Total Demand")
netDemandTrace = go.Scatter(x=xValues, y=yNetDemand, name="Net Demand")
solarSupplyTrace = go.Scatter(x=xValues, y=ySolarSupply, name="Solar")
windSupplyTrace = go.Scatter(x=xValues, y=yWindSupply, name="Wind")
coalSupplyTrace = go.Scatter(x=xValues, y=yCoalSupply, name="Coal")
gasSupplyTrace = go.Scatter(x=xValues, y=yGasSupply, name="Natural Gas")

# Set chart title
hourlyFigure.update_layout(title_text="March 1, 2020 - Hourly Load and Selected Energy Sources",
                        paper_bgcolor='#F5F6F9',
                        plot_bgcolor='rgba(0,0,0,0)')
# Set x-axis
hourlyFigure.update_xaxes(nticks=8)
hourlyFigure.update_xaxes(title_text="Time of Day", gridcolor='#DEDEDE')
# Set y-axes
hourlyFigure.update_yaxes(title_text="MW", secondary_y=False, gridcolor='#DEDEDE')

# Add traces to chart
hourlyFigure.add_traces([demandTrace, netDemandTrace, solarSupplyTrace, windSupplyTrace, coalSupplyTrace, gasSupplyTrace])

There are two concerns with the duck-like shape of the **Net Demand** curve that limit how much solar energy can be added to the grid.

The first is that there is a limit to how deep the 'belly' of the Net Demand curve can fall. On March 1, 2020, solar energy had to be [curtailed](http://www.caiso.com/Documents/Wind_SolarReal-TimeDispatchCurtailmentReportMar01_2020.pdf) &mdash; mostly due to transmission facilities not having necessary capacity to deliver low-cost solar to where it needs to go &mdash; from 9 am to 4 pm. The maximum rate of curtailment was just over 1,000 MWh in the hour after noon. This amount was not unusual for California's grid system in March, but there's more to it. Even if transmission lines were capable of shifting solar energy supply to where it's needed there is only so much more that Net Demand can be reduced before baseline natural gas resources have to be shut off and then turned on again, which causes higher [Non-Fuel Operational and Maintenance](https://www.scottmadden.com/insight/californias-combined-cycle-costs-age-duck-curve/) (NFOM) costs.

The second is that the largest source for renewable energy in California, solar, cannot provide power at peak demand between 6:30 pm and 10 pm due to lack of sunlight. Replacing fossil fuel sourced electricty with that of renewables will require shifting power from intermittent sources, like solar, to evenings.

***

The National Renewable Energy Laboratory proposes [two approaches](https://www.nrel.gov/news/program/2018/10-years-duck-curve.html) to tackling these issues.

The first is to "fatten" the duck by growing the "belly" of the duck. In other words, making operational changes that allow natural gas plants to cycle more frequently with reduced NFOM costs. This gives grid operators more flexibility to shut down natural gas energy during the daytime on particularly sunny days.

The second solution is "flatten" the duck, shift supply from solar from the daytime to the evening using energy storage &mdash; such as grid-scale batteries &mdash; or shift demand from the evening to the afternoon using demand response, which involves paying consumers to not consume electricity at peak load times.

***

There is another concern that I wanted to explore, which is that natural gas power plant cycling should lead to more CO<sub>2</sub> emissions in the first hour or so of the evening. This would be the result of Combined Cycle baseload power plants forced to run a "simple cycle" after being turned off until they generate enough heat in their exhaust to be able to boil water into steam and turn a steam turbine in addition to the gas powered main turbine. This allows a natural gas power plant to jump from around 40% efficiency to around 60%. However, the hot start-up time &mdash; the time it takes to reach the plant's full capacity when the plant has been shutdown for fewer than 8 hours &mdash; can be around [40 minutes](https://www.sciencedirect.com/science/article/pii/S1364032117309206) for a combined cycle gas plant.

Perhaps there is an increase in the rate of emissions per MW from natural gas as a result of the plants running a simple cycle first before reaching combined cycle efficiencies.


### <a name="carbon-emissions"></a> Carbon Emissions

Fortunately, CAISO also provides emissions data for carbon-emitting sources of electricity.

In [3]:
# Get data as arrays
yTotalCO2 = fullDf['Total CO2'].tolist()
yImportsCO2 = fullDf['Imports_CO2'].tolist()
yGasCO2 = fullDf['Natural gas_CO2'].tolist()
yBiogasCO2 = fullDf['Biogas_CO2'].tolist()
yBiomassCO2 = fullDf['Biomass_CO2'].tolist()
yGeothermalCO2 = fullDf['Geothermal_CO2'].tolist()
yCoalCO2 = fullDf['Coal_CO2'].tolist()

# Create figure with secondary y-axis
emissionsFigure = make_subplots()
    
# Create traces
totalCo2Trace = go.Scatter(x=xValues, y=yTotalCO2, name="Total CO\u2082")
importsCO2Trace = go.Scatter(x=xValues, y=yImportsCO2, name="Imports CO\u2082")
gasCO2Trace = go.Scatter(x=xValues, y=yGasCO2, name="Natural Gas CO\u2082")
biogasCO2Trace = go.Scatter(x=xValues, y=yBiogasCO2, name="Biogas CO\u2082")
biomassCO2Trace = go.Scatter(x=xValues, y=yBiomassCO2, name="Biomass CO\u2082")
geothermalCO2Trace = go.Scatter(x=xValues, y=yGeothermalCO2, name="Geothermal CO\u2082")
coalCO2Trace = go.Scatter(x=xValues, y=yCoalCO2, name="Coal CO\u2082")

# Set chart title
emissionsFigure.update_layout(title_text="March 1, 2020 - California Grid Hourly Emissions",
                        paper_bgcolor='#F5F6F9',
                        plot_bgcolor='rgba(0,0,0,0)')
# Set x-axis
emissionsFigure.update_xaxes(nticks=8)
emissionsFigure.update_xaxes(title_text="Time of Day", gridcolor='#DEDEDE')
# Set y-axes
emissionsFigure.update_yaxes(title_text="mTCO\u2082/h", gridcolor='#DEDEDE')

# Add traces to chart
emissionsFigure.add_traces([totalCo2Trace, importsCO2Trace, gasCO2Trace, \
                            biogasCO2Trace, biomassCO2Trace, geothermalCO2Trace, coalCO2Trace])

Unsurprisingly, the emissions data from CAISO matches the duck curve pattern shown in the hourly load data.

#### <a name="natural-gas-peak-emissions"></a> Unexpected result in rising Natural Gas and Natural Gas CO<sub>2</sub> emissions per MW

What was surprising was efficiency of natural gas power plants as they ramp up and down during the day.

In [4]:
yGasCo2PerMw = []
for i in range(len(xValues)):
    yGasCo2PerMw.append(yGasCO2[i] / yGasSupply[i])

# Create figure with secondary y-axis
gasFigure = make_subplots(specs=[[{"secondary_y": True}]])
    
# Create traces
gasSupplyTrace = go.Scatter(x=xValues, y=yGasSupply, name="Natural Gas Power")
gasCo2PerMwTrace = go.Scatter(x=xValues, y=yGasCo2PerMw, name="Gas CO\u2082 / MWh")

# Set chart title
gasFigure.update_layout(title_text="Natural Gas Efficiency",
                        paper_bgcolor='#F5F6F9',
                        plot_bgcolor='rgba(0,0,0,0)')
# Set x-axis
gasFigure.update_xaxes(nticks=8)
gasFigure.update_xaxes(title_text="Time of Day", gridcolor='#DEDEDE')
# Set y-axes
gasFigure.update_yaxes(title_text="MW", secondary_y=False, gridcolor='#DEDEDE')
gasFigure.update_yaxes(title_text="mTCO\u2082 / MWh", secondary_y=True, showgrid=False)

# Add traces to chart
gasFigure.add_traces([gasSupplyTrace, gasCo2PerMwTrace], secondary_ys=[False, True])

I was surprised by this chart. I had expected to see a rise in emissions per MW during the ramp up. Surprisingly CAISO's data shows that producing more electricity from natural gas improves the efficiency of natural gas used to supply the grid.

One possible explanation is that some natural gas power plant operators are shifting from combined cycle to simple cycle power production during the day to reduce their production and follow load demand. Then, in the evening they switch back to combined cycle and produce electricity more efficiently.

Unfortunately, the dataset provided by CAISO is not detailed enough to examine individual power plant production decisions. In any case the grid operator's dependence on natural gas to supply peak load still emits a large amount of CO<sub>2</sub> that could be reduced by battery storage, demand response, and improved energy efficiency.

### <a name="electric-vehicles"></a> Charging More Electric Vehicles
In my recent [article](https://medium.com/age-of-awareness/energy-efficiency-should-be-publicized-more-than-renewables-7e5e2192d0d6) I argued that energy efficiency can play a crucial role in shrinking the "duck's head" and that electric vehicles should be a cause for concern given that a fleet of more clean electric vehicles on the road could increase the scale of peak load during the evening when commuters return home.

In [5]:
fig = make_subplots(specs=[[{"secondary_y": True}]])

# Create traces
demandT = go.Scatter(x=xValues, y=yDemand, name="Demand (MW)")
netDemandT = go.Scatter(x=xValues, y=yNetDemand, name="Net Demand (MW)")
gasCO2Trace = go.Scatter(x=xValues, y=yGasCO2, name="Natural Gas CO\u2082")
importsCo2Trace = go.Scatter(x=xValues, y=yImportsCO2, name="Imports CO\u2082")

fig.add_traces([demandT, netDemandT, gasCO2Trace, importsCo2Trace], secondary_ys=[False, False, True, True])
evFigure = go.FigureWidget(fig)

I'm going to chart the total **Demand** and **Net Demand** curves along with the Natural Gas and Import (electricity provided by other states from a combination of carbon-heavy and renewable sources) CO<sub>2</sub> emission curves. We'll add some simple sliders to see how CO<sub>2</sub> emissions increase as a result of:

- the number of additional EVs on the road
- the start time for when commuters begin charging their EVs
- how long EVs spend charging during peak load hours

Some assumptions that we will make include:
- The charging rate of an electric vehicle. It varies greatly based on the charging setup and the vehicle itself.  We'll assume that most EV owners will opt to charge at home using 240v / 32A chargers that charge at a rate of 7.7 kWh and &mdash; depending on the vehicle &mdash; will result in roughly [25 miles charged per hour](https://www.clippercreek.com/charging-times-chart/).
- How much a commuter needs to charge every night.
    The average Californian commutes [40.1. miles](https://www.answerfinancial.com/insurance-center/which-states-have-the-longest-commute/?a=insurance-quote) per day.
    
Based on these assumptions we will set a default charging time of 40.1 miles / 25 miles per hour = 1.572 hours or roughly 1 hr 34 minutes of charging time.

In [6]:
import ipywidgets as widgets
from datetime import datetime, timedelta

# Create widgets
evNumber = widgets.IntSlider(value=0, min=0, max=5000, step=100, continuous_update=True)

evStartCharging = widgets.SelectionSlider(options=evFigure.data[0].x, value='17:00', continuous_update=True)

evChargeDuration = widgets.SelectionSlider(options=evFigure.data[0].x[0:100], # roughly 8 hr range
                                           value='01:35', continuous_update=True)
# Create Output
out = widgets.Output(layout={'border': '1px solid black'})
out.append_stdout('Move sliders to see additional CO\u2082 emissions. ')

# Handle changes to input widgets
def response(change):
    startIndex = evFigure.data[1].x.index(evStartCharging.value)
    startTime = timedelta(hours = int(evStartCharging.value[0:2]), minutes = int(evStartCharging.value[3:5]))
    addtlTime = timedelta(hours = int(evChargeDuration.value[0:2]), minutes = int(evChargeDuration.value[3:5]))
    endIndex = evFigure.data[1].x.index((datetime.min + (startTime + addtlTime)).time().strftime("%H:%M")) # evStartCharging.value + evChargeDuration.value)

    newDemand = fullDf['Demand (5 min. avg.)'].tolist()
    newNetDemand = fullDf['Net demand'].tolist()
    oldGasEmission = fullDf['Natural gas_CO2'].tolist()
    newGasEmission = fullDf['Natural gas_CO2'].tolist()
    oldImportEmission = fullDf['Imports_CO2'].tolist()
    newImportEmission = fullDf['Imports_CO2'].tolist()
    additionalGasCO2 = 0
    additionalImportsCO2 = 0
    
    for i in range(len(fullDf.index)):
        if i >= startIndex and i < endIndex:
            additionalDemandMW = evNumber.value * 1000 * .0077
            newDemand[i] = newDemand[i] + additionalDemandMW
            newNetDemand[i] = newNetDemand[i] + additionalDemandMW
            newGasEmission[i] = newGasEmission[i] + additionalDemandMW * yGasCo2PerMw[i]
            newImportEmission[i] = newImportEmission[i] + additionalDemandMW * 0.428 # see below
            # calculate area of additional emissions in 5 minute intervals
            additionalGasCO2 = additionalGasCO2 + (newGasEmission[i] - oldGasEmission[i]) * (1/12)
            additionalImportsCO2 = additionalImportsCO2 + (newImportEmission[i] - oldImportEmission[i]) * (1/12)
    
    with out:
        out.clear_output()
        out.append_stdout('Amount of additional electricity demand: ' \
                          + str(round(additionalDemandMW, 2)) \
                          + '. Amount of additional CO\u2082 emitted: ' \
                          + str(round(additionalGasCO2 + additionalImportsCO2, 2)) \
                          + ' mTCO\u2082. Equivalent to annual emissions from ' \
                          + str(round(additionalGasCO2 + additionalImportsCO2 / 8.67, 2)) \
                          + ' U.S. homes. \n')
        
    with evFigure.batch_update():
        evFigure.data[0].y = newDemand
        evFigure.data[1].y = newNetDemand
        evFigure.data[2].y = newGasEmission
        evFigure.data[3].y = newImportEmission

# Activate widget listners
evNumber.observe(response, names="value")
evStartCharging.observe(response, names="value")
evChargeDuration.observe(response, names="value")

How do we quantify the additional power supplied to charge electric vehicles?

We can use the federal Environmental Protection Agency's equivalency calculator to get an idea for how much additional electricity is needed. The EPA states that the average U.S. home consumes [8.67 metric tons CO<sub>2</sub> consumed per home per year](https://www.epa.gov/energy/greenhouse-gases-equivalencies-calculator-calculations-and-references), which we can use to calculate how many homes could be powered by this additional CO2 release.

When calculating the additional CO<sub>2</sub> emissions from charging electric vehicles we will use:
- The CO<sub>2</sub> emissions per MW for additional natural gas power
- The California Air Resources Board unspecified [emission rate](https://www.caiso.com/Documents/GreenhouseGasEmissionsTracking-Methodology.pdf) of 0.428 mTCO<sub>2</sub> / MWh for EIM (Energy Imbalance Market) imports to support ISO load

In [7]:
# Set chart title
evFigure.update_layout(title_text="March 1, 2020 - Additional Electricity Demand from Electric Vehicles",
                       paper_bgcolor='#F5F6F9',
                       plot_bgcolor='rgba(0,0,0,0)')
# Set x-axis title
evFigure.update_xaxes(nticks=8)
evFigure.update_xaxes(title_text="Time of Day", gridcolor='#DEDEDE')
# Set y-axes titles
evFigure.update_yaxes(title_text="MW", secondary_y=False, gridcolor='#DEDEDE')
evFigure.update_yaxes(title_text="mTCO\u2082 / h", secondary_y=True, showgrid=False)

# Setup Layout and Show
labelsContainer = widgets.HBox(
    children = [widgets.Label('Number of additional EVs (in thousands):', layout = widgets.Layout(width='33%')),
                widgets.Label('Start time for EV charging:', layout = widgets.Layout(width='33%')),
                widgets.Label('Duration of charge time:', layout = widgets.Layout(width='33%'))])
widgetsContainer = widgets.HBox(
    children = [evNumber, evStartCharging, evChargeDuration],
    layout = widgets.Layout(justify_content='space-between'))
widgets.VBox([labelsContainer, widgetsContainer, evFigure, out])

#### <a name="ev-grid-release"></a> How do Additional EVs Affect Grid CO<sub>2</sub> Release

It is unsurprising that additional electric vehicles will add additional demand to California's grid. But it may be surprising how much. If an additional 600,000 electric vehicles are added in California &mdash; doubling the current [600,000](https://www.sfchronicle.com/climate/article/Californians-are-buying-up-electric-cars-But-14447810.php) plug-in EVs in California, which represent roughly 4% of the [15 million](https://www.statista.com/statistics/196010/total-number-of-registered-automobiles-in-the-us-by-state/) registered vehicles in Califonia &mdash; the CO<sub>2</sub> emissions from 3,735 U.S. homes will be released if they charge for 1 hr 35 minutes at 5 pm.

In addition, the grid will need to supply an additional 4,620 MW of electricity, which is almost a 20% increase over total electricity demand at this time. That is a **substantial** increase for the grid to accomodate and the state of California [has a goal of](https://www.cpuc.ca.gov/zev/) *5 million* Zero-Emission Vehicles by 2030. That would *double* the amount of electricity needed to supply California's grid at peak times.

#### <a name="approach-grid-demand"></a> How to Approach Additional Grid Demand From EVs

Now it must be emphasized that this chart does not tell us how many CO<sub>2</sub> emissions are *saved* from replacing internal combustion engine vehicles with zero-emission vehicles. But, if ignoring the CO<sub>2</sub> emissions still leaves the problem of a substantial increase in electricity demand. There are several approaches that can limit the threat to grid stability.

1) Energy efficiency is a way to reduce demand throughout the day but especially in the evening.

2) **Demand Response** programs can shift demand from peak hours to times when there are less demand and electricity is cheap. Electric vehicle owners could be incentivized with small payments to set their cars to charge after midnight or during weekend daytime hours &mdash; when they may not need to leave the home and can charge with grid solar &mdash; saving them money and helping the grid operator meet demand.

3) Time-of-use (TOU) utility plans can incentivize EV owners to charge during the daytime. Today, most California EV owners pay for high-usage tier electricty, but [one-quarter](https://www.synapse-energy.com/sites/default/files/EV-Impacts-June-2019-18-122.pdf) of them pay Time-of-use rates. Incentivizing EV owners to charge outside of peak hours can "flatten" the belly of the duck curve, which allows more solar to be introduced to the grid. In fact, in the interactive chart the 600,000 additional EVs could be shifted to noon instead of 5 pm, and additional demand would not exceed March 1's peak load.

### <a name="takeaways"></a> Takeaways

The previous model is simplistic in many ways but can help explain how increasing adoption of EVs creates challenges.

More than that, it also provides opportunities. Incentivizing EV owners to charge their vehicles during the day also means extra demand that additional solar can provide. Increasingly affordable battery storage can supply EV charging in the evening and limit the use of natural gas and CO<sub>2</sub> emissions during peak hours.

In the short-term the goal is to integrate more renewable energy sources &mdash; reducing CO<sub>2</sub> emissions &mdash; and solar can continue to provide additional capacity when combined with economic incentives for consumers.